In [107]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('train.csv')
data.head()

0.24236252545824846

## Data Pre-processing

In [85]:
df = data.copy()

# Extract surnames from the "Name" column
df['Surname'] = df['Name'].apply(lambda x: x.split(',')[0].strip())

# Create a new column for family size
df['Family_Size'] = df.groupby('Surname')['Surname'].transform('count')

# Add 1 to include the person with that surname itself
df['Family_Size'] = df['Family_Size'] + 1


# Encode the Cabin Type as a Feature and One-hot encode Sex and Embarked columns
df['CabinType'] = df['Cabin'].str[0]
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'CabinType'], dtype=int)

# Also include a column representing if a passeneger has a column
df['HasCabin'] = np.where(df['Cabin'].isna(), 0, 1)


# Age column has several NaN values. Augment these with median age instead of removing
median_age = df['Age'].median()
df['Age'].fillna(median_age, inplace=True)


# Extract titles from the "Name" column
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.')

# One-hot encode the titles which occur three or more times into features
titles_to_encode = ['Mr', 'Miss', 'Mrs', 'Master', 'Dr', 'Rev']

for title in titles_to_encode:
    df['Title_' + title] = (df['Title'] == title).astype(int)
    

# Drop irrelevant columns
df.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Title', 'Surname'], axis=1, inplace=True)



# Step 1: Split the data into training and test sets
X = df.drop(columns=['Survived'])  # Features (all columns except 'Survived')
y = df['Survived']  # Target variable

# Split the data into 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Apply Standard Scaling
scaler = StandardScaler()

# Fit the scaler on the training data and transform both the training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



## Train Classification Models

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC()
}

# Train and evaluate each classifier
for name, classifier in classifiers.items():
    classifier.fit(X_train_scaled, y_train)  # Train the model
    y_pred = classifier.predict(X_test_scaled)  # Make predictions on the test set
    accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
    print(f'{name} Accuracy: {accuracy:.2f}')

Logistic Regression Accuracy: 0.80
Decision Tree Accuracy: 0.79
Random Forest Accuracy: 0.83
SVM Accuracy: 0.81


## Tuning Best Model

In [87]:
# Split the data into 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Apply Standard Scaling
scaler = StandardScaler()

# Fit the scaler on the training data and transform both the training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Random Forest model on the entire feature set
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_scaled, y_train)

# Evaluate the model on the test set
y_pred = rf_classifier.predict(X_test_scaled)
initial_accuracy = accuracy_score(y_test, y_pred)
print(f'Initial Accuracy: {initial_accuracy:.2f}')

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Rank features by importance
feature_ranking = pd.Series(feature_importances, index=X_train.columns).sort_values(ascending=False)

# Select the top K features
K = 20  # Replace with the desired number of top features
selected_features = feature_ranking[:K].index.tolist()

# Retrain the Random Forest model using only the selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Fit the scaler on the training data and transform both the training and test data
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

rf_classifier_selected = RandomForestClassifier(random_state=42)
rf_classifier_selected.fit(X_train_scaled, y_train)

# Evaluate the model with selected features on the test set
y_pred_selected = rf_classifier_selected.predict(X_test_scaled)
selected_accuracy = accuracy_score(y_test, y_pred_selected)
print(f'Accuracy with {K} selected features: {selected_accuracy:.2f}')

Initial Accuracy: 0.83
Accuracy with 20 selected features: 0.84


## Grid Search

In [88]:
from sklearn.model_selection import GridSearchCV

# Define a parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest model
rf_classifier = RandomForestClassifier(random_state=42)

# Create an instance of GridSearchCV
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV instance on the training data
grid_search.fit(X_train_scaled, y_train)

# Retrieve the best hyperparameters and the corresponding best estimator
best_params = grid_search.best_params_
best_rf_classifier = grid_search.best_estimator_

# Evaluate the model with the best hyperparameters on the test set
y_pred_best = best_rf_classifier.predict(X_test_scaled)
best_accuracy = accuracy_score(y_test, y_pred_best)

print(f'Best Hyperparameters: {best_params}')
print(f'Accuracy with Best Hyperparameters: {best_accuracy:.2f}')

Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 10}
Accuracy with Best Hyperparameters: 0.84


## Train Best Model

In [101]:
scaler = StandardScaler()

# Retrain the Random Forest model using only the selected features
X_train_selected = X_train[selected_features]

# Fit the scaler on the training data and transform both the training and test data
X_train_scaled = scaler.fit_transform(X_train_selected)


# Create a Random Forest model
final_rf_classifier = RandomForestClassifier(random_state=42, max_depth = 10, min_samples_leaf =  2, min_samples_split = 5, n_estimators = 10)

# Fit the Model
final_rf_classifier.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=10, random_state=42)

In [102]:
test_df = pd.read_csv('test.csv')

df = test_df.copy()

# Extract surnames from the "Name" column
df['Surname'] = df['Name'].apply(lambda x: x.split(',')[0].strip())

# Create a new column for family size
df['Family_Size'] = df.groupby('Surname')['Surname'].transform('count')

# Add 1 to include the person with that surname itself
df['Family_Size'] = df['Family_Size'] + 1


# Encode the Cabin Type as a Feature and One-hot encode Sex and Embarked columns
df['CabinType'] = df['Cabin'].str[0]
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'CabinType'], dtype=int)

# Also include a column representing if a passeneger has a column
df['HasCabin'] = np.where(df['Cabin'].isna(), 0, 1)


# Age column has several NaN values. Augment these with median age instead of removing
median_age = df['Age'].median()
df['Age'].fillna(median_age, inplace=True)

# Fare column has several NaN values. Augment these with median fare instead of removing
median_fare = df['Fare'].median()
df['Fare'].fillna(median_fare, inplace=True)


# Extract titles from the "Name" column
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.')

# One-hot encode the titles which occur three or more times into features
titles_to_encode = ['Mr', 'Miss', 'Mrs', 'Master', 'Dr', 'Rev']

for title in titles_to_encode:
    df['Title_' + title] = (df['Title'] == title).astype(int)
    

# Drop irrelevant columns
df.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Title', 'Surname'], axis=1, inplace=True)

# Select features and scale unseen data
X_test = df.copy()  # Features (all columns except 'Survived')
X_test_selected = X_test[selected_features]
X_test_scaled = scaler.transform(X_test_selected)

# Evaluate the model with the best hyperparameters on the test set
preds = final_rf_classifier.predict(X_test_scaled)

# Create Data Frame with Passenger ID and Prediction
final_predictions = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': preds})

final_predictions.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [103]:
# Save predictions as csv file
final_predictions.to_csv('predictions.csv', index=False)
